<a href="https://colab.research.google.com/github/bobovnii/progressive_growing_of_gans/blob/master/GANs_Progressive_Growing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
 from google.colab import drive
 drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
!ls /gdrive/My\ Drive/

ls: cannot access '/gdrive/My Drive/': No such file or directory


In [3]:
!pwd

/content


In [0]:
import os
os.chdir("/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans")

In [0]:
!git clone https://github.com/bobovnii/progressive_growing_of_gans.git

Cloning into 'progressive_growing_of_gans'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 350.22 KiB | 8.34 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [21]:
!ls

 config.py				      misc.py
 dataset				      networks.py
 dataset.py				      __pycache__
 dataset_tool.py			      README.md
 img_align_celeba_png			      representative_image_512x256.png
'karras2018iclr-celebahq-1024x1024 (1).pkl'   requirements-pip.txt
 karras2018iclr-celebahq-1024x1024.pkl	      results
 karras2018iclr-lsun-airplane-256x256.pkl     testPictures
 legacy.py				      tfutil.py
 LICENSE.txt				      train.py
 loss.py				      util_scripts.py
 metrics


In [6]:
%tensorflow_version 1.x
import tensorflow as tf
tf.VERSION

'1.15.0'

In [7]:
import tensorflow-gpu as tf2
tf2.VERSION

SyntaxError: ignored

In [0]:
#pip install tensorflow-gpu==1.6.0

In [5]:
#pip install -r requirements-pip.txt

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [8]:
import pickle
import numpy as np
import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import PIL.Image

# Initialize TensorFlow session.
tf.InteractiveSession()

# Import official CelebA-HQ networks.
with open('karras2018iclr-celebahq-1024x1024.pkl', 'rb') as file:
    G, D, Gs = pickle.load(file)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
# Generate latent vectors.
latents = np.random.RandomState(1000).randn(1000, *Gs.input_shapes[0][1:]) # 1000 random latents
latents = latents[[477, 56, 83, 887, 583, 391, 86, 340, 341, 415]] # hand-picked top-10

In [0]:
# Generate dummy labels (not used by the official networks).
labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])

In [0]:
# Run the generator to produce a set of images.
images = Gs.run(latents, labels)

In [0]:
# Convert images to PIL-compatible format.
images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) # [-1,1] => [0,255]
images = images.transpose(0, 2, 3, 1) # NCHW => NHWC

In [0]:
# Save images as PNG.
for idx in range(images.shape[0]):
    PIL.Image.fromarray(images[idx], 'RGB').save('img%d.png' % idx)

In [0]:
import config
import tfutil
#import dataset
#import dataset_tool 
import pickle


In [0]:
!mkdir dataset

In [0]:
!ls ./../../dataset/CelebA

CelebA	deltas00000.zip


In [0]:
!python dataset_tool.py create_celeba dataset/CelebA ./../../dataset/CelebA

Loading CelebA from "./../../dataset/CelebA"
Error: Expected to find 51333 images


In [0]:
!ls ./../../dataset/CelebA/Img/img_align_celeba_png.7z/

img_align_celeba_png.7z.001  img_align_celeba_png.7z.009
img_align_celeba_png.7z.002  img_align_celeba_png.7z.010
img_align_celeba_png.7z.003  img_align_celeba_png.7z.011
img_align_celeba_png.7z.004  img_align_celeba_png.7z.012
img_align_celeba_png.7z.005  img_align_celeba_png.7z.013
img_align_celeba_png.7z.006  img_align_celeba_png.7z.014
img_align_celeba_png.7z.007  img_align_celeba_png.7z.015
img_align_celeba_png.7z.008  img_align_celeba_png.7z.016


In [0]:
!unzip -d ./../../dataset/CelebA/Img/ ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.*

Archive:  ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001 or
        ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001.zip, and cannot find ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001.ZIP, period.


In [0]:
!7z x ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./../../dataset/CelebA/Img/img_align_celeba_png.7z/                                                             1 file, 734003200 bytes (700 MiB)

Extracting archive: ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001
  0% 1 Open           --
Path = ./../../dataset/CelebA/Img/img_align_celeba_png.7z/img_align_celeba_png.7z.001
Type = Split
Physical Size = 734003200
Volumes = 16
Total Physical Size = 11586962073
----
Path = img_align_celeba_png.7z
Size = 11586962073
--
Path = img_align_celeba_png.7z
Type = 7z
Physical Size = 11586962073
Headers Size = 1977786
Method = LZMA:24
Solid

In [0]:
!ls

config.py	      LICENSE.txt  README.md
dataset		      loss.py	   representative_image_512x256.png
dataset.py	      metrics	   requirements-pip.txt
dataset_tool.py       misc.py	   tfutil.py
img_align_celeba_png  networks.py  train.py
legacy.py	      __pycache__  util_scripts.py


In [0]:
!pwd

/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/img_align_celeba_png


In [0]:
ls img_align_celeba_png/ | wc -l 

ls: cannot open directory 'img_align_celeba_png/': Input/output error
0


In [0]:
os.chdir("img_align_celeba_png/")

In [0]:
!ls *.png| wc -l

ls: cannot access '*.png': Input/output error
0


In [0]:
!python dataset_tool.py create_celeba dataset/CelebA /gdrive/My\ Drive/Colab\ Notebooks/GAN/progressive_growing_of_gans/

Loading CelebA from "/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/"
Creating dataset "dataset/CelebA"
W0722 19:41:32.339983 140443047040896 deprecation_wrapper.py:119] From dataset_tool.py:71: The name tf.python_io.TFRecordOptions is deprecated. Please use tf.io.TFRecordOptions instead.

W0722 19:41:32.340302 140443047040896 deprecation_wrapper.py:119] From dataset_tool.py:71: The name tf.python_io.TFRecordCompressionType is deprecated. Please use tf.compat.v1.python_io.TFRecordCompressionType instead.

W0722 19:41:32.340493 140443047040896 deprecation_wrapper.py:119] From dataset_tool.py:74: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

Added 6355 images.
Traceback (most recent call last):
  File "dataset_tool.py", line 738, in <module>
    execute_cmdline(sys.argv)
  File "dataset_tool.py", line 733, in execute_cmdline
    func(**vars(args))
  File "dataset_tool.py", line 447, in create_celeba
    assert img.shape ==

In [0]:
import glob
aaa = sorted(glob.glob(os.path.join('/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/img_align_celeba_png', '*')))
len(aaa)

51333

In [0]:
import numpy as np
import tensorflow as tf
import PIL.Image

In [0]:
    img = np.asarray(PIL.Image.open(aaa[0]))
    resolution = img.shape[0]
    channels = img.shape[2] if img.ndim == 3 else 1
    img.shape[0]
    img.shape[1] 

178

In [0]:
!python dataset_tool.py extract dataset/CelebA/ testPictures

Loading dataset "dataset/CelebA/"




2019-09-24 20:27:11.543512: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-24 20:27:11.543775: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x210f480 executing computations on platform Host. Devices:
2019-09-24 20:27:11.543840: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Extracting images to "testPictures"
2019-09-24 20:27:15.546670: W tensorflow/co

In [0]:
os.chdir("/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans")

In [0]:
!mkdir testPictures

In [0]:
!python train.py

Initializing TensorFlow...




Running train.train_progressive_gan()...
Streaming data using dataset.TFRecordDataset...
Traceback (most recent call last):
  File "train.py", line 285, in <module>
    tfutil.call_func_by_name(**config.train)
  File "/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/tfutil.py", line 236, in call_func_by_name
    return import_obj(func)(*args, **kwargs)
  File "/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/train.py", line 2447, in train_progressive_gan
    training_set = dataset.load_dataset(data_dir=config.data_dir, verbose=True, **config.dataset)
  File "/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/dataset.py", line 235, in load_dataset
    dataset = tfutil.import_obj(class_name)(**adjusted_kwargs)
  File "/gdrive/My Drive/Colab Notebooks/GAN/progressive_growing_of_gans/dataset.py", line 67, in __init__
    print(elf.tfrecord_dir)
NameError: name 'elf' is not defined
